<a href="https://colab.research.google.com/github/lexoz-bedra/probability_and_statistics_labs/blob/main/Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задача 4


## Условие

Рассмотрите схемы Бернулли при 𝑛 ∈ {100, 1000, 10000} и
𝑝 ∈ {0.001, 0.01, 0.1, 0.25, 0.5} и рассчитайте точные вероятности (где это возможно)
P(𝑆𝑛 ∈ [𝑛/2 −
√𝑛𝑝𝑞, 𝑛/2 + √𝑛𝑝𝑞]), P(𝑆𝑛 ⩽ 5) и максимальную вероятность вида P(𝑆𝑛 = 𝑘), 𝑆𝑛
– количество успехов в 𝑛 испытаниях, и приближенные с помощью одной из предельных теорем.
Сравните точные и приближенные вероятности. Объясните результаты.

## Решение

Рассмотрим схемы Бернулли для заданных значений 𝑛 и 𝑝, посчитаем для них следующие вероятности:

- P(𝑆𝑛 ∈ [𝑛/2 − √𝑛𝑝𝑞, 𝑛/2 + √𝑛𝑝𝑞]) (`interval_probability`)
- P(𝑆𝑛 ⩽ 5) (`under6_probability`)
- Максимальную вероятность вида P(𝑆𝑛 = 𝑘) (`max_probability`)

Точные вероятности будем считать с помощью формулы Бернулли (там, где это посчитается, не выдаст ошибку и не округлит очень маленькие числа до нуля), а приближённые - с помощью локальной предельной теоремы Муавра-Лапласа или теоремы Пуассона, в зависимости от того, какая теорема будет применима в конкретном случае.

Для работы с большими числами будем использовать библиотеку gmpy2, несмотря на то, что совсем маленькие числа она округлит до нуля (если этого не делать и полагаться на встроенную в питон длинную арифметику, то рано или поздно, при 𝑛 = 10000, всё равно возникнет ошибка).

In [ ]:
!pip install gmpy2

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import sys
import gmpy2
import math
import numpy as np
import pandas as pd

import scipy.stats


sys.set_int_max_str_digits(100000)

### Точные вероятности по формуле Бернулли

In [ ]:
# точное вычисление вероятности по формуле Бернулли
def calculation1(n, k, probability_of_success):
    C = gmpy2.comb(n, k)
    return (C * probability_of_success**k * (1 - probability_of_success)**(n - k))

In [ ]:
# находим границы интервала [𝑛/2 − √𝑛𝑝𝑞, 𝑛/2 + √𝑛𝑝𝑞]
def find_interval(n, p):
    q = 1 - p
    left = math.ceil(n / 2 - math.sqrt(n * p * q))
    right = math.floor(n / 2 + math.sqrt(n * p * q))
    return left, right


# считаем точную вероятность на интервале
def accurate_interval_probability(n, p):
    left, right = find_interval(n, p)
    probability = 0
    for i in range(left, right + 1):
        probability += calculation1(n, i, p)
    return probability


# точная вероятность P(𝑆𝑛 ⩽ 5)
def accurate_under6_probability(n, p):
    left, right = find_interval(n, p)
    probability = 0
    for i in range(0, 6):
        probability += calculation1(n, i, p)
    return probability


# максимальная из точных вероятностей P(𝑆𝑛 = 𝑘)
'''def accurate_max_probability(n, p):
    max_probability = 0
    index_max_probability = 2
    for i in range(0, n + 1):
        probability = calculation1(n, i, p)
        if probability > max_probability:
            max_probability = probability
            index_max_probability = i
    return index_max_probability, max_probability
'''

# индекс максимальной вероятности
def accurate_max_probability_k(n, p):
    index_max_probability = round(n * p)
    max_probability = calculation1(n, index_max_probability, p)
    return index_max_probability


# максимальная из точных вероятностей P(𝑆𝑛 = 𝑘)
def accurate_max_probability(n, p):
    index_max_probability = round(n * p)
    max_probability = calculation1(n, index_max_probability, p)
    return max_probability

In [ ]:
# результаты точных вычислений
n_values = [100, 1000, 10000]
p_values = [0.001, 0.01, 0.1, 0.25, 0.5]

df_acc = pd.DataFrame(columns=["n", "p", "accurate_interval_probability", "accurate_under6_probability",
                               "accurate_max_probability_k", "accurate_max_probability"])

for n in n_values:
    for p in p_values:
        accurate_interval_prob = accurate_interval_probability(n, p)
        accurate_under6_prob = accurate_under6_probability(n, p)
        max_prob = accurate_max_probability(n, p)

        df_acc = df_acc.append({"n": n, "p": p, "accurate_interval_probability": accurate_interval_prob,
                        "accurate_under6_probability": accurate_under6_prob,
                        "accurate_max_probability_k": max_prob[0], "accurate_max_probability": max_prob[1]}, ignore_index=True)

df_acc


Как мы видим, при 𝑛 = 10000 результаты некоторых вычислений превращаются в нули. Будем считать, что в этих случаях точную вероятность посчитать невозможно, а ещё она слишком мала, чтобы быть значимой.

### Приближённые вероятности

Перед подсчётом приближённой вероятности выберем, какую теорему будем использовать, посчитав произведение 𝑛𝑝.

- 𝑛𝑝 < 1  - теорема Пуассона
- 𝑛𝑝 >=1 - теоремы Муавра-Лапласа (локальная или интегральная)

In [ ]:
def laplasa_myavra_local_calculation(n, k, probability_of_success):
    if n == 0 or k == 0:
        return 0
    x_k = (k - n * probability_of_success) / math.sqrt(k * n * probability_of_success)
    result = 1 / (math.sqrt(k * n * probability_of_success)) * math.exp(-0.5 * x_k * x_k)
    return result


def laplasa_myavra_interval_calculation(n, k1, k2, p):
    if n == 0 or k1 == 0:
        return 0
    q = 1 - p
    x_1 = (k1 - n * p) / math.sqrt(n * p * q)
    x_2 = (k2 - n * p) / math.sqrt(n * p * q)

    cdf_x_1 = scipy.stats.norm.cdf(x_1, loc=0, scale=1) - 0.5
    cdf_x_2 = scipy.stats.norm.cdf(x_2, loc=0, scale=1) - 0.5

    return cdf_x_2 - cdf_x_1


def poisson_calculation(n, k, p):
    lambda_ = n * p
    log_probability = k * np.log(lambda_) - lambda_ - np.sum(np.log(range(1, k+1)))
    probability = np.exp(log_probability)
    return probability

In [ ]:
# находим границы интервала [𝑛/2 − √𝑛𝑝𝑞, 𝑛/2 + √𝑛𝑝𝑞]
def find_interval(n, p):
    q = 1 - p
    left = math.ceil(n / 2 - math.sqrt(n * p * q))
    right = math.floor(n / 2 + math.sqrt(n * p * q))
    return left, right


# считаем приближённую вероятность на интервале
def approximate_interval_probability(n, p):
    left, right = find_interval(n, p)
    return laplasa_myavra_interval_calculation(n, left, right, p)



# приближённая вероятность P(𝑆𝑛 ⩽ 5)
def approximate_under6_probability(n, p):
    return laplasa_myavra_interval_calculation(n, 0, 5, p)

"""
# максимальная из приближённых вероятностей P(𝑆𝑛 = 𝑘)
def approximate_max_probability(n, p):
    max_probability = 0
    index_max_probability = 2
    for i in range(0, n + 1):
        probability = laplasa_myavra_calculation(n, i, p)
        if probability > max_probability:
            max_probability = probability
            index_max_probability = i
    return index_max_probability, max_probability
"""

# индекс максимальной вероятности
def approximate_max_probability_k(n, p):
    index_max_probability = round(n * p)
    max_probability = laplasa_myavra_local_calculation(n, index_max_probability, p)
    return index_max_probability


# максимальная из приближённых вероятностей P(𝑆𝑛 = 𝑘)
def approximate_max_probability(n, p):
    index_max_probability = round(n * p)
    max_probability = laplasa_myavra_local_calculation(n, index_max_probability, p)
    return max_probability

In [ ]:
# вероятность P(𝑆𝑛 ∈ [𝑛/2 − √𝑛𝑝𝑞, 𝑛/2 + √𝑛𝑝𝑞])
def poisson_interval_probability(n, p):
    q = 1 - p
    left = n / 2 - math.sqrt(n * p * q)
    right = n / 2 + math.sqrt(n * p * q)

    probability = sum(poisson_calculation(n, k, p) for k in range(int(left), int(right) + 1))

    return probability


# вероятность P(𝑆𝑛 ⩽ 5)
def poisson_under6_probability(n, p):
    q = 1 - p
    left = n / 2 - math.sqrt(n * p * q)
    right = n / 2 + math.sqrt(n * p * q)

    probability = sum(poisson_calculation(n, k, p) for k in range(6))

    return probability

# индекс для максимальной вероятности
def poisson_max_probability_k(n, p):
    max_prob = 0
    index_max_prob = 0
    for k in range(n + 1):
        probability = poisson_calculation(n, k, p)
        if probability > max_prob:
            max_prob = probability
            index_max_prob = k
    return index_max_prob

"""
# максимальная вероятность вида P(𝑆𝑛 = 𝑘)
def poisson_max_probability(n, p):
    max_prob = 0
    index_max_prob = 0
    for k in range(n + 1):
        probability = poisson_calculation(n, k, p)
        if probability > max_prob:
            max_prob = probability
            index_max_prob = k
    return max_prob
"""

# максимальная вероятность вида P(𝑆𝑛 = 𝑘)
def poisson_max_probability(n, p):
    index_max_prob = round(n * p)
    probability = poisson_calculation(n, index_max_prob, p)
    return probability

In [ ]:
# выбираем, по какой теореме будем считать вероятность

def calculate_probability(n, p):
    if n * p < 1:
        return (poisson_interval_probability(n, p),
                poisson_under6_probability(n, p),
                poisson_max_probability_k(n, p),
                poisson_max_probability(n, p))
    else:
        return (approximate_interval_probability(n, p),
                approximate_under6_probability(n, p),
                approximate_max_probability_k(n, p),
                approximate_max_probability(n, p))

In [ ]:
# результаты вычислений для сравнения
n_values = [100, 1000, 10000]
p_values = [0.001, 0.01, 0.1, 0.25, 0.5]

results = []
for n in n_values:
    for p in p_values:
        approximate_interval_prob, approximate_under6_prob, approximate_max_k, \
        approximate_max_prob = calculate_probability(n, p)
        results.append((n, p,
                        accurate_interval_probability(n, p), approximate_interval_prob,
                        accurate_under6_probability(n, p), approximate_under6_prob,
                        accurate_max_probability_k(n, p),
                        accurate_max_probability(n, p), approximate_max_k, approximate_max_prob))

df = pd.DataFrame(results, columns=["n", "p",
                                    "accurate_interval_probability", "approximate_interval_probability",
                                    "accurate_under6_probability", "approximate_under6_probability",
                                    "accurate_max_probability_k", "accurate_max_probability",
                                    "approximate_max_probability_k", "approximate_max_probability"])
df

,n,p,accurate_interval_probability,approximate_interval_probability,accurate_under6_probability,approximate_under6_probability,accurate_max_probability_k,accurate_max_probability,approximate_max_probability_k,approximate_max_probability
0,100,0.001,9.5968414768106632e-122,2.975060e-115,0.99999999890018931,1.0,0,0.90479214711370892,0,0.904837
1,100,0.010,6.1039875571729981e-72,0.000000e+00,0.99946546553600613,0.0,1,0.36972963764972644,1,1.000000
2,100,0.100,3.6123083280873955e-21,0.000000e+00,0.057576886487033956,0.0,10,0.13186534682448858,10,0.100000
3,100,0.250,4.2504440254864535e-06,6.180994e-07,1.1700149154089138e-07,0.0,25,0.091799691766836791,25,0.040000
4,100,0.500,0.72874697592616533,6.826895e-01,6.2616225626926802e-23,0.0,50,0.079589237387178768,50,0.020000
5,1000,0.001,0.0,0.000000e+00,0.99941192989823624,0.0,1,0.36806348825922292,1,1.000000
6,1000,0.010,0.0,0.000000e+00,0.0661395116072514,0.0,10,0.12574021112620629,10,0.100000
7,1000,0.100,0.0,0.000000e+00,2.5565456930600531e-38,0.0,100,0.042016790861086603,100,0.010000
8,1000,0.250,1.4970596601027823e-58,0.000000e+00,3.9691404522735886e-115,0.0,250,0.029124105883705082,250,0.004000
9,1000,0.500,0.67306331756840454,6.572183e-01,7.7385053062943519e-289,0.0,500,0.0252250181783608,500,0.002000


Результаты приближений достаточно точные, но в каких-то случаях погрешность всё же довольно большая. Это может быть связано с тем, что предельные теоремы применяются на большом (стремящемся к бесконечности) количестве испытаний, и чем оно больше, тем точнее измерения.